In [1]:
!pwd

/root/ensemble_commit/10


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

df = pd.read_json(r'../datasets/patch_db.json', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'non-security':0,'security':1}
df = df.replace({"category": label2id})
df

train, test = train_test_split(df, test_size=0.3, random_state=42)
test, val = train_test_split(test, test_size=0.5, random_state=42)
train,_ = train_test_split(train, train_size=0.1, random_state=42)
test, val = train_test_split(test, train_size=0.1, random_state=42)
val, _ = train_test_split(val, train_size=0.1, random_state=42)

train.reset_index(inplace=True)
test.reset_index(inplace=True)
val.reset_index(inplace=True)

/tmp/ipykernel_5515/3916909791.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"category": label2id})


In [4]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['diff_code'])
y_train = train['category']

# 3. 训练决策树分类器
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# 4. 评估模型
y_pred = clf.predict(vectorizer.transform(test['diff_code']))
print("Accuracy:", accuracy_score(test['category'], y_pred))
print("Classification Report:\n", classification_report(test['category'], y_pred, digits=4))

Accuracy: 0.6554934823091247
Classification Report:
               precision    recall  f1-score   support

           0     0.7635    0.7243    0.7434       370
           1     0.4516    0.5030    0.4759       167

    accuracy                         0.6555       537
   macro avg     0.6076    0.6137    0.6097       537
weighted avg     0.6665    0.6555    0.6602       537



In [5]:
from joblib import load,dump

In [6]:
dump(clf, 'DT_patchDB.joblib')

['DT_patchDB.joblib']